In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
from urllib.request import urlopen, Request

In [2]:
r = Request('https://th.investing.com/economic-calendar/', headers={'User-Agent': 'Mozilla/5.0'})
time.sleep(2)
response = urlopen(r).read()
soup = BeautifulSoup(response, "html.parser")
table = soup.find_all(class_ = "js-event-item")
result = []

In [3]:
def event_calendar():
    
    for bl in table:
        event_datetime = bl.get('data-event-datetime', '').strip()
        currency = bl.find(class_="left flagCur noWrap").text.strip()
        intensity_divs = bl.find_all(class_="left textNum sentiment noWrap")
        event = bl.find(class_="left event").text.strip()
        intencity_val = 0
        true_count = 0

        for intence in intensity_divs:
            _true = intence.find_all(class_="grayFullBullishIcon")
            _false = intence.find_all(class_="grayEmptyBullishIcon")

            true_count = len(_true)

            if true_count == 3:
                intencity_val = 3
            elif true_count == 2:
                intencity_val = 2
            else :
                intencity_val = 1
                
        event_datetime = event_datetime.split(' ')
        date=event_datetime[0]
        time=event_datetime[1]
        
        
        result.append({'currency' : currency, 'date' : date, 'time' : time, 'intensity' : intencity_val, 'event':event})

    return result

In [4]:
news = event_calendar()
news_df = pd.DataFrame(news)

In [5]:
display(news_df.shape)
display(news_df.head(10))

(77, 5)

,currency,date,time,intensity,event
0,USD,2025/05/22,00:30:00,2,การประมูลพันธบัตรอายุ 20 ปี
1,BRL,2025/05/22,00:30:00,1,ดัชนีกระแสการแลกเปลี่ยนเงินตราต่างประเทศของบราซิล
2,AUD,2025/05/22,06:00:00,1,ดัชนี PMI ภาคการผลิต ( พ.ค.)
3,AUD,2025/05/22,06:00:00,1,ดัชนี PMI ภาคบริการ ( พ.ค.)
4,JPY,2025/05/22,06:50:00,1,ยอดคำสั่งซื้อเครื่องจักรกล (ปีต่อปี) (มี.ค.)
5,JPY,2025/05/22,06:50:00,1,ยอดคำสั่งซื้อสินค้าประเภทเครื่องจักร (เดือนต่อ...
6,JPY,2025/05/22,06:50:00,1,ยอดการซื้อตราสารหนี้ต่างประเทศ
7,JPY,2025/05/22,06:50:00,1,การลงทุนจากต่างประเทศในหุ้นญี่ปุ่น
8,SGD,2025/05/22,07:00:00,1,ดัชนีจีดีพีของสิงคโปร์ (ปีต่อปี) (ไตรมาส 1)
9,SGD,2025/05/22,07:00:00,1,ดัชนีจีดีพีของสิงคโปร์ (ไตรมาสต่อไตรมาส) (ไตร...


In [6]:
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
news_df.to_csv(f'Data/Event_Calendar/{date_time}.csv', index=False)

In [10]:
print(news_df[news_df['intensity'] == 3][['currency', 'event']])

   currency                                              event
56      USD        จำนวนคนที่ยื่นขอรับสวัสดิการว่างงานครั้งแรก
64      USD  ดัชนีผู้จัดการฝ่ายจัดซื้อ (PMI) ภาคการผลิต  ( ...
66      USD  ดัชนีผู้จัดการฝ่ายจัดซื้อ (PMI) ภาคการบริการ  ...
67      USD    ยอดขายบ้านมือสอง (Existing Home Sales)  (เม.ย.)
